<a href="https://colab.research.google.com/github/pank8/python/blob/master/Sellout_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import feather
import re
from sklearn.metrics.pairwise import cosine_similarity
try:
    from mlxtend.frequent_patterns import apriori
    from mlxtend.preprocessing import TransactionEncoder
    from jellyfish import jaro_winkler
except:
    print('**************************\n\nУ вас не установлены некоторые библиотеки, это не проблема - мы сделаем дальше все руками\n\n**************************\n')
from scipy import sparse

%matplotlib inline

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
pd.set_option('display.float_format','{:,.3f}'.format)

l = [['Š', 'S'],['š', 's'],['Ð', 'Dj'],['Ž', 'Z'],['ž', 'z'],
     ['À', 'A'],['Á', 'A'],['Â', 'A'],['Ã', 'A'],['Ä', 'A'],
     ['Å', 'A'],['Æ', 'A'],['Ç', 'C'],['È', 'E'],['É', 'E'],
     ['Ê', 'E'],['Ë', 'E'],['Ì', 'I'],['Í', 'I'],['Î', 'I'],
     ['Ï', 'I'],['Ñ', 'N'],['Ń', 'N'],['Ò', 'O'],['Ó', 'O'],
     ['Ô', 'O'],['Õ', 'O'],['Ö', 'O'],['Ø', 'O'],['Ù', 'U'],
     ['Ú', 'U'],['Û', 'U'],['Ü', 'U'],['Ý', 'Y'],['Þ', 'B'],
     ['ß', 'Ss'],['à', 'a'],['á', 'a'],['â', 'a'],['ã', 'a'],
     ['ä', 'a'],['å', 'a'],['æ', 'a'],['ç', 'c'],['è', 'e'],
     ['é', 'e'],['ê', 'e'],['ë', 'e'],['ì', 'i'],['í', 'i'],
     ['î', 'i'],['ï', 'i'],['ð', 'o'],['ñ', 'n'],['ń', 'n'],
     ['ò', 'o'],['ó', 'o'],['ô', 'o'],['õ', 'o'],['ö', 'o'],
     ['ø', 'o'],['ù', 'u'],['ú', 'u'],['û', 'u'],['ü', 'u'],
     ['ý', 'y'],['ý', 'y'],['þ', 'b'],['ÿ', 'y'],['ƒ', 'f'],
     ['ă', 'a'],['î', 'i'],['â', 'a'],['ș', 's'],['ț', 't'],
     ['Ă', 'A'],['Î', 'I'],['Â', 'A'],['Ș', 'S'],['Ț', 'T']]

l1 = [i[0] for i in l]
l2 = [i[1] for i in l]

chars_to_normal_map = {key:value for key,value in zip(l1,l2)}

def convert_to_latin(text):
    if type(text) == str:
        cross_chars = (set(chars_to_normal_map.keys()).intersection(text))
        if len(cross_chars) > 0:
            for char in cross_chars:
                text = text.replace(char,chars_to_normal_map[char])
    return text


def remove_punctuation(text,join_by = ' '):
    text = text.replace('S/A','S.A')
    
    punc = list(r' !"#$&%\'()*,-/:;<=>?@[\\]^_`{|}~№«»“ʹ”’¤¶�¢®´–')
    nopunc = []
    for char in text:
        if char in ['&','+']:
            nopunc.append(' and ')
            
        if (char in punc):
            nopunc.append(' ')
        elif char == '.':
            nopunc.append('') 
        else:
            nopunc.append(char)
    text = ''.join(nopunc)
    text = text.split()
    if len(''.join(text)) > 2:
        text = f'{join_by}'.join(text)
    else:
        return np.nan
    
    return text

**************************

У вас не установлены некоторые библиотеки, это не проблема - мы сделаем дальше все руками

**************************



In [0]:
distrubutor_data = \
pd.read_excel("Malaysian comparison e2open sdh customers 2019 10 07.xlsx",
              sheet_name = 'E2Open MALAYSIA ')

our_data = \
pd.read_excel("Malaysian comparison e2open sdh customers 2019 10 07.xlsx",
              sheet_name = 'SDH MALAYSIA')

In [0]:
# Заменяем (null) и пустые стринги на np.nan
# Отличие в том, что '' - не NaN, поэтому при визуализации мы их не увидим как пустые

for each in ['(null)','',' ']:
    distrubutor_data.replace(each,np.nan,inplace = True)
    our_data.replace(each,np.nan,inplace = True)

In [0]:
# Сохраняем название колонок в переменные
# Так мы можем не запоминать, как во внешней базе был указан адрес,
# а написать лишь external address
# Плюс от базы к базе можем использовать один и тот же код, 
# указав лишь названия колонок один раз в начале

external_name = 'E2Open Name'
external_city = 'E2Open City'
external_address = 'E2Open Address'
external_zip = 'E2Open Postal'
external_id = 'E2Open Id'
external_index = 'indexX'

internal_name = 'golden_sold_to_name'
internal_city = 'golden_sold_to_city'
internal_address = 'golden_sold_to_address'
internal_zip = 'golden_sold_to_postal_code'
internal_id = 'golden_sold_to_id'
internal_index = 'indexY'

In [0]:
internal_data = pd.DataFrame(our_data.reset_index().rename(columns = {'index':'indexY'})[[internal_index,
                                                                                          internal_id,
                                                                                          internal_name,
                                                                                          internal_city,
                                                                                          internal_address,
                                                                                          internal_zip]],
                             copy = True)

external_data = pd.DataFrame(distrubutor_data.reset_index().rename(columns = {'index':'indexX'})[[external_index,
                                                                                                  external_id,
                                                                                                  external_name,
                                                                                                  external_city,
                                                                                                  external_address,
                                                                                                  external_zip]],
                             copy = True)

# 0. Drop Empty Names

As we can't match them

In [7]:
internal_data.loc[internal_data[internal_name].isnull(),:]

,indexY,golden_sold_to_id,golden_sold_to_name,golden_sold_to_city,golden_sold_to_address,golden_sold_to_postal_code
4348,4348,154,NaN,NaN,NaN,NaN


In [0]:
internal_data = pd.DataFrame(internal_data.drop(internal_data.loc[internal_data['golden_sold_to_name'].isnull(),:].index).reset_index().drop('index',axis = 1))

In [10]:
external_data.loc[external_data[external_name].isnull(),:]

,indexX,E2Open Id,E2Open Name,E2Open City,E2Open Address,E2Open Postal


In [0]:
external_data = pd.DataFrame(external_data.drop(external_data.loc[external_data['E2Open Name'].isnull(),:].index).reset_index().drop('index',axis = 1))

### Check

In [0]:
external_data['Working String'] = external_data[external_name].apply(str) + ' '\
+ external_data[external_zip].apply(str)

internal_data['Working String'] = internal_data[internal_name].apply(str) + ' '\
+ internal_data[internal_zip].apply(str)

In [13]:
internal_data['Working String'].iloc[0]

'(SAINS) Sarawak Information System Sdn Bhd 93502'

In [14]:
internal_data.loc[internal_data['Working String'] == 'ABLECON POWER SYSTEM SDN BHD 40150',:]

,indexY,golden_sold_to_id,golden_sold_to_name,golden_sold_to_city,golden_sold_to_address,golden_sold_to_postal_code,Working String
49,49,0037067895,ABLECON POWER SYSTEM SDN BHD,Shah Alam,"5, Jalan Pemberita U1/49, Temasya Industrial P...",40150,ABLECON POWER SYSTEM SDN BHD 40150


In [0]:
external_data['Working String'] = external_data[external_name].apply(str) + ' '\
+ external_data[external_zip].apply(str)

internal_data['Working String'] = internal_data[internal_name].apply(str) + ' '\
+ internal_data[internal_zip].apply(str)

print(len(external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]))
external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]

230 Intersections

Let's try make it better by lowercasing the Working String

In [16]:
external_data['Working String'] = external_data['Working String'].str.lower()

internal_data['Working String'] = internal_data['Working String'].str.lower()

print(len(external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]))
external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]

329


,indexX,E2Open Id,E2Open Name,E2Open City,E2Open Address,E2Open Postal,Working String
20,20,102506423,ABLECON POWER SYSTEM SDN BHD,SHAH ALAM,5 jalan pemberita u1 49 temasya indl park,40150,ablecon power system sdn bhd 40150
25,25,102497423,ACCESS ELECTRICAL SDN BHD,TAWAU,LOT 2 MILE 5 1 2 JALAN APAS 0,91000,access electrical sdn bhd 91000
35,35,103001272,ACS CONCEPT (M) SDN BHD,SHAH ALAM,47 jalan g u8 g bukit jelutong,40150,acs concept (m) sdn bhd 40150
49,49,101242879,ADVANCE LITE ELECTRICAL SDN BHD,PERAI,10 tgk kikik 7 grd flr tmn inderawasih,13600,advance lite electrical sdn bhd 13600
57,57,102705260,ADVANCED MICRO DEVICES GLOBAL SERVICES (M) SDN...,BAYAN LEPAS,units 2 3 17 24 lower 6th flr jalan bukit jambul,11900,advanced micro devices global services (m) sdn...
...,...,...,...,...,...,...,...
3880,3880,102746582,WYWY ELECTRICAL ENGINEERING,PUCHONG,b 1 26 kompleks suria kinrara persiaran kinrara,47100,wywy electrical engineering 47100
3921,3921,101688130,YEW LIANG ELECTRICAL WORKS,KUCHING,no 159 padungan rd,93100,yew liang electrical works 93100
3958,3958,101234997,YONG SHEN HARDWARE (SEL ) SDN BHD,SHAH ALAM,2 20g jalan pahat g15 g dataran otomobil seksy...,40200,yong shen hardware (sel ) sdn bhd 40200
3984,3984,101236074,YUZIN M&E SDN BHD,PETALING JAYA,"NO2, JALAN SS25/35, TAMAN MAYANG,",47301,yuzin m&e sdn bhd 47301


We find 329 companies

Let's see if the number will change after the next steps

# City shouldn't be used - Use Zip Code instead

# 1. Zip Cleaning

In [0]:
internal_data['Clean' + internal_zip] = internal_data[internal_zip].apply(lambda s: remove_punctuation(str(s),join_by = ''))
external_data['Clean' + external_zip] = external_data[external_zip].apply(lambda s: remove_punctuation(str(s),join_by = ''))

In [0]:
internal_data[internal_zip] = internal_data[internal_zip].apply(lambda s: remove_punctuation(str(s),join_by = ''))
external_data[external_zip] = external_data[external_zip].apply(lambda s: remove_punctuation(str(s),join_by = ''))

In [0]:
internal_data = internal_data[[internal_index,internal_id,internal_name,internal_city,internal_address,internal_zip]]
external_data = external_data[[external_index,external_id,external_name,external_city,external_address,external_zip]]

In [21]:
external_data['Working String'] = external_data[external_name].astype(str).str.lower() + ' '\
+ external_data[external_zip].astype(str).str.lower()

internal_data['Working String'] = internal_data[internal_name].astype(str).str.lower() + ' '\
+ internal_data[internal_zip].astype(str).str.lower()

print(len(external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]))
external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]

337


,indexX,E2Open Id,E2Open Name,E2Open City,E2Open Address,E2Open Postal,Working String
20,20,102506423,ABLECON POWER SYSTEM SDN BHD,SHAH ALAM,5 jalan pemberita u1 49 temasya indl park,40150,ablecon power system sdn bhd 40150
25,25,102497423,ACCESS ELECTRICAL SDN BHD,TAWAU,LOT 2 MILE 5 1 2 JALAN APAS 0,91000,access electrical sdn bhd 91000
35,35,103001272,ACS CONCEPT (M) SDN BHD,SHAH ALAM,47 jalan g u8 g bukit jelutong,40150,acs concept (m) sdn bhd 40150
49,49,101242879,ADVANCE LITE ELECTRICAL SDN BHD,PERAI,10 tgk kikik 7 grd flr tmn inderawasih,13600,advance lite electrical sdn bhd 13600
57,57,102705260,ADVANCED MICRO DEVICES GLOBAL SERVICES (M) SDN...,BAYAN LEPAS,units 2 3 17 24 lower 6th flr jalan bukit jambul,11900,advanced micro devices global services (m) sdn...
...,...,...,...,...,...,...,...
3921,3921,101688130,YEW LIANG ELECTRICAL WORKS,KUCHING,no 159 padungan rd,93100,yew liang electrical works 93100
3958,3958,101234997,YONG SHEN HARDWARE (SEL ) SDN BHD,SHAH ALAM,2 20g jalan pahat g15 g dataran otomobil seksy...,40200,yong shen hardware (sel ) sdn bhd 40200
3984,3984,101236074,YUZIN M&E SDN BHD,PETALING JAYA,"NO2, JALAN SS25/35, TAMAN MAYANG,",47301,yuzin m&e sdn bhd 47301
3993,3993,102939802,ZENBI SDN BHD,SUBANG JAYA,ss 15,47500,zenbi sdn bhd 47500


337 Exact Matches after Zip Cleaning

# 2. Clean Names

In [0]:
internal_data[internal_name] = internal_data[internal_name].apply(convert_to_latin)
external_data[external_name] = external_data[external_name].apply(convert_to_latin)

internal_data[internal_name] = internal_data[internal_name].apply(remove_punctuation)
external_data[external_name] = external_data[external_name].apply(remove_punctuation)

In [0]:
internal_data = internal_data[[internal_index,internal_id,internal_name,internal_city,internal_address,internal_zip]]
external_data = external_data[[external_index,external_id,external_name,external_city,external_address,external_zip]]

In [24]:
internal_data.loc[internal_data[internal_name].isnull(),:]

,indexY,golden_sold_to_id,golden_sold_to_name,golden_sold_to_city,golden_sold_to_address,golden_sold_to_postal_code


In [25]:
external_data.loc[external_data[external_name].isnull(),:]

,indexX,E2Open Id,E2Open Name,E2Open City,E2Open Address,E2Open Postal


In [0]:
ids_to_drop = internal_data.loc[internal_data[internal_name].isnull(),:].index
internal_data = pd.DataFrame(internal_data.drop(ids_to_drop).reset_index().drop('index',axis = 1))

ids_to_drop = external_data.loc[external_data[external_name].isnull(),:].index
external_data = pd.DataFrame(external_data.drop(ids_to_drop).reset_index().drop('index',axis = 1))

In [0]:
internal_data = internal_data[[internal_index,
                               internal_id,
                               internal_name,
                               internal_city,
                               internal_address,
                               internal_zip]]

external_data = external_data[[external_index,
                               external_id,
                               external_name,
                               external_city,
                               external_address,
                               external_zip]]

In [28]:
corpus_internal = {}

for each in internal_data[internal_name].dropna():
    each = each.lower()
    each = each.split()
    for word in each:
        if word in corpus_internal:
            corpus_internal[word] += 1
        else:
            corpus_internal[word] = 1
            
corpus_internal = pd.Series(corpus_internal,name = 'Internal').sort_values(ascending = False)
corpus_internal = corpus_internal[corpus_internal > len(internal_data)*0.005]
corpus_internal = corpus_internal.reset_index().rename(columns = {'index':'word'}).reset_index().rename(columns = {'index':'rank_in'}).set_index('word')
corpus_internal['rank_in'] += 1
corpus_internal.head(10)

,rank_in,Internal
word,,
bhd,1,4872
sdn,2,4715
malaysia,3,1101
m,4,667
and,5,461
berhad,6,451
engineering,7,409
services,8,244
electrical,9,185


In [29]:
corpus_external = {}

for each in external_data[external_name].dropna():
    each = each.lower()
    each = each.split()
    for word in each:
        if word in corpus_external:
            corpus_external[word] += 1
        else:
            corpus_external[word] = 1
            
corpus_external = pd.Series(corpus_external,name = 'External').sort_values(ascending = False)
corpus_external = corpus_external[corpus_external > len(external_data)*0.005]
corpus_external = corpus_external.reset_index().rename(columns = {'index':'word'}).reset_index().rename(columns = {'index':'rank_ex'}).set_index('word')
corpus_external['rank_ex'] += 1
corpus_external.head(10)

,rank_ex,External
word,,
sdn,1,2709
bhd,2,2706
electrical,3,846
and,4,461
trading,5,407
engineering,6,376
m,7,303
hardware,8,243
electric,9,211


In [0]:
corpus = pd.concat([corpus_external,
           corpus_internal],
          axis = 1,
          sort = False)

In [31]:
pd.set_option('display.float_format','{:.0f}'.format)
corpus

,rank_ex,External,rank_in,Internal
sdn,1,2709,2,4715
bhd,2,2706,1,4872
electrical,3,846,9,185
and,4,461,5,461
trading,5,407,47,59
engineering,6,376,7,409
m,7,303,4,667
hardware,8,243,nan,nan
electric,9,211,26,89
enterprise,10,202,55,53


In [0]:
stopwords = ['sdn','bhd','berhad','of','co','company','ltd','syarikat','pty','pte','corporation']

def remove_stopwords(text):
    text = text.lower()
    text = text.split()
    for stopword in stopwords:
        if stopword in text:
            text.remove(stopword)
    return ' '.join(text)

In [0]:
internal_data[internal_name] = internal_data[internal_name].apply(remove_stopwords)
external_data[external_name] = external_data[external_name].apply(remove_stopwords)

In [35]:
external_data['Working String'] = external_data[external_name].apply(str) + ' '\
+ external_data[external_zip].apply(str)

internal_data['Working String'] = internal_data[internal_name].apply(str) + ' '\
+ internal_data[internal_zip].apply(str)

print(len(external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]))
external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:]

428


,indexX,E2Open Id,E2Open Name,E2Open City,E2Open Address,E2Open Postal,Working String
12,12,101234673,a tech electrical,PUCHONG,no 32 jalan utama 1 12 taman perindustrial puc...,47100,a tech electrical 47100
20,20,102506423,ablecon power system,SHAH ALAM,5 jalan pemberita u1 49 temasya indl park,40150,ablecon power system 40150
25,25,102497423,access electrical,TAWAU,LOT 2 MILE 5 1 2 JALAN APAS 0,91000,access electrical 91000
35,35,103001272,acs concept m,SHAH ALAM,47 jalan g u8 g bukit jelutong,40150,acs concept m 40150
38,38,101259071,actgen industry,JOHOR BAHRU,133 jln seroja 39 taman johor jaya,81100,actgen industry 81100
...,...,...,...,...,...,...,...
3921,3921,101688130,yew liang electrical works,KUCHING,no 159 padungan rd,93100,yew liang electrical works 93100
3958,3958,101234997,yong shen hardware sel,SHAH ALAM,2 20g jalan pahat g15 g dataran otomobil seksy...,40200,yong shen hardware sel 40200
3984,3984,101236074,yuzin m and e,PETALING JAYA,"NO2, JALAN SS25/35, TAMAN MAYANG,",47301,yuzin m and e 47301
3993,3993,102939802,zenbi,SUBANG JAYA,ss 15,47500,zenbi 47500


In [36]:
len(external_data)

4005

428 Exact Matches ~ 10% Of External Companies are found

In [0]:
# Сохраняем Exact Matches в отдельную таблицу

matches_exact = pd.merge(external_data.loc[external_data['Working String'].isin(internal_data['Working String']),:],
                         internal_data.loc[internal_data['Working String'].isin(external_data['Working String']),:],
                         how = 'outer',
                         on = 'Working String')

In [38]:
matches_exact

,indexX,E2Open Id,E2Open Name,E2Open City,E2Open Address,E2Open Postal,Working String,indexY,golden_sold_to_id,golden_sold_to_name,golden_sold_to_city,golden_sold_to_address,golden_sold_to_postal_code
0,12,101234673,a tech electrical,PUCHONG,no 32 jalan utama 1 12 taman perindustrial puc...,47100,a tech electrical 47100,29,0015657466,a tech electrical,Puchong,"No 32A, Jalan Utama 1/12, Taman Perindustrian ...",47100
1,20,102506423,ablecon power system,SHAH ALAM,5 jalan pemberita u1 49 temasya indl park,40150,ablecon power system 40150,49,0037067895,ablecon power system,Shah Alam,"5, Jalan Pemberita U1/49, Temasya Industrial P...",40150
2,25,102497423,access electrical,TAWAU,LOT 2 MILE 5 1 2 JALAN APAS 0,91000,access electrical 91000,53,0037610890,access electrical,"TAWAU, SABAH","LOT.2, MILE 5 1/2 , JALAN APAS,",91000
3,25,102497423,access electrical,TAWAU,LOT 2 MILE 5 1 2 JALAN APAS 0,91000,access electrical 91000,391,0003594060,access electrical,TAWAU,"TB 4007, BERKELEY PARK,¶APAS ROAD, MILES 1.5",91000
4,35,103001272,acs concept m,SHAH ALAM,47 jalan g u8 g bukit jelutong,40150,acs concept m 40150,64,0001933617,acs concept m,Shah Alam,"10 (GROUND FLOOR), JALAN ASTAKA L (U8/L), SEKS...",40150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,3921,101688130,yew liang electrical works,KUCHING,no 159 padungan rd,93100,yew liang electrical works 93100,7486,0001937938,yew liang electrical works,KUCHING,"NO 159, JALAN PADUNGAN ,",93100
459,3958,101234997,yong shen hardware sel,SHAH ALAM,2 20g jalan pahat g15 g dataran otomobil seksy...,40200,yong shen hardware sel 40200,7500,0036783581,yong shen hardware sel,Shah Alam,"2-20G,Jalan Pahat G15/G,Dataran Otomobil",40200
460,3984,101236074,yuzin m and e,PETALING JAYA,"NO2, JALAN SS25/35, TAMAN MAYANG,",47301,yuzin m and e 47301,7521,0035184468,yuzin m and e,PETALING JAYA,"No 2, Jalan ss25/35, taman mayang",47301
461,3993,102939802,zenbi,SUBANG JAYA,ss 15,47500,zenbi 47500,7566,0041311716,zenbi,"Subang Jaya, Selangor Darul Ehsan","First Subang, N-08-02, Jalan SS15/4G",47500


# 2. Matching

In [0]:
# Удаляем indexX и indexY тех записей, которые мы уже заматчили

internal_data = internal_data.drop(internal_data.loc[internal_data[internal_index].isin(matches_exact[internal_index]),:].index)
external_data = external_data.drop(external_data.loc[external_data[external_index].isin(matches_exact[external_index]),:].index)

internal_data['Working String'] = internal_data['Working String'].apply(lambda s: s.replace('\xad',''))
internal_data['Working String'] = internal_data['Working String'].apply(lambda s: s.replace('\u00ad',''))
internal_data['Working String'] = internal_data['Working String'].apply(lambda s: s.replace('\N{SOFT HYPHEN}',''))

In [0]:
def shingle_tfo(text,shingle_size = 2):
    text = ''.join(text.split())
    result = []

    for counter in range(0,len(text) - shingle_size + 1):
        result.append(text[counter:counter + shingle_size])
    result = list(np.unique(result))
    
    return result

In [0]:
# Оставляем только Working String и indexX/indexY
internal_table = internal_data[[internal_index,'Working String']].reset_index().drop('index',axis = 1)
external_table = external_data[[external_index,'Working String']].reset_index().drop('index',axis = 1)

In [43]:
internal_table.head(2)

,indexY,Working String
0,0,sains sarawak information system 93502
1,1,0000311608 package 4b 89850


In [44]:
external_table.head(2)

,indexX,Working String
0,0,1 living depot 81100
1,1,1 stop led supplies m 47810


In [0]:
# Добавляем колонку с разбитыми Working String

internal_table['Split'] = internal_table['Working String'].apply(shingle_tfo)
external_table['Split'] = external_table['Working String'].apply(shingle_tfo)

In [0]:
all_working_strings = pd.concat([internal_table['Split'],
                                 external_table['Split']]).reset_index()['Split']

In [0]:
try:
    te = TransactionEncoder()
    matrix = te.fit_transform(all_working_strings,sparse = True)

    matrix_in = matrix[:len(internal_table)]
    matrix_ex = matrix[len(internal_table):]
except:
    print('К сожалению у вас не получится воспользоваться Transaction Encoder, поэтому код ниже - замена ему')
    unique_shingles = {}

    for each_record in all_working_strings.index:
        value = all_working_strings[each_record]
        for item in value:
            if item not in unique_shingles:
                unique_shingles[item] = [each_record]
            else:
                unique_shingles[item].append(each_record)
                
    print(f'{len(unique_shingles)} уникальных шинглов (колонок в нашей матрице)')
    
    # Теперь для каждой записи по каждому биграмму проставим 1 или 0 в зависимости от наличия
    
    # На выходе имеем таблицу, где по каждому биграму выведены записи,
    # в которых он присутствует
    pd.Series(unique_shingles).head()
    
    df = pd.DataFrame(index = all_working_strings.index)
    counter = 1
    quarter = len(unique_shingles)//4

    for keyword in unique_shingles:
        records = unique_shingles[keyword]
        df.loc[records,keyword] = 1
        counter += 1
        if counter % quarter == 0:
            print(f'{counter * 100//base}% Done')
            
            
    matrix = sparse.csc_matrix(df.fillna(0))
    matrix_in = matrix[:len(internal_table)]
    matrix_ex = matrix[len(internal_table):]

In [0]:
similar_accs = {}
counter = 0
threshold = 0.9

for each in matrix_ex:
    df = pd.Series(cosine_similarity(each,matrix_in)[0])
    df = df[df > threshold]
    if len(df) > 0:
        similar_accs[counter] = (df.index.tolist())
        
    counter += 1
    
similar_accs = pd.Series(similar_accs,name = f'{threshold*100}% similar IDs')
similar_accs = pd.DataFrame(similar_accs)

In [49]:
# Для каждой записи в external_table, для которой существуеют матчи во внутренней таблице
# получаем 90% схожие записи в internal_table
similar_accs.head()

,90.0% similar IDs
122,[249]
137,[280]
270,[811]
290,[712]
381,[939]


In [0]:
similar_accs = similar_accs.reset_index().rename(columns = {'index':'Record Number',f'{threshold*100}% similar IDs':'indexYs'})

In [0]:
record_to_index = {key:value for key,value in zip(external_table.loc[similar_accs['Record Number'],:].index,
                                external_table.loc[similar_accs['Record Number'],'indexX'])}

In [0]:
similar_accs['indexX'] = similar_accs['Record Number'].map(record_to_index)

In [53]:
similar_accs.head()

,Record Number,indexYs,indexX
0,122,[249],144
1,137,[280],162
2,270,[811],315
3,290,[712],337
4,381,[939],436


In [54]:
record = 0

print(external_table.loc[similar_accs.loc[record,'Record Number'],'Working String'])
print()
print(similar_accs.loc[record,:])
print()
internal_table.loc[similar_accs.loc[similar_accs['Record Number'] == similar_accs.loc[record,'Record Number'],'indexYs'].iloc[0]]

antara steel mills 81700

Record Number      122
indexYs          [249]
indexX             144
Name: 0, dtype: object



,indexY,Working String,Split
249,266,antara steel mills 81707,"[07, 17, 70, 81, an, ar, as, ee, el, il, ll, l..."


In [0]:
d = {}

for each in similar_accs[['indexX','indexYs']].apply(lambda x: {i:x[0] for i in x[1]},axis = 1).values:
    for each_key in each:
        d[internal_table.loc[each_key,'indexY']] = each[each_key]

In [0]:
internal_data.loc[internal_data[internal_index].isin(d),external_index] = internal_data.loc[internal_data[internal_index].isin(d),internal_index].map(d)

In [0]:
semi_matches = pd.merge(external_data,
                         internal_data,
                         how = 'inner',
                         on = external_index)

In [58]:
semi_matches.head(2)

,indexX,E2Open Id,E2Open Name,E2Open City,E2Open Address,E2Open Postal,Working String_x,indexY,golden_sold_to_id,golden_sold_to_name,golden_sold_to_city,golden_sold_to_address,golden_sold_to_postal_code,Working String_y
0,144,102922885,antara steel mills,PASIR GUDANG,kawasan perindustrian pasir gudang,81700,antara steel mills 81700,266,0001972722,antara steel mills,Pasir Gudang,"PLO 277, JLN GANGSA 1 P.O. BOX 79,PASIR GUDANG...",81707,antara steel mills 81707
1,162,102497443,arrowmedia,SHAH ALAM,NO 1 JALAN ANGGERIK VANILLA AG 31 AG SEKSYEN 3...,40460,arrowmedia 40460,300,0001921736,arrow media,SHAH ALAM,"22G, Jalan Anggerik Vanilla W31/W Seksyen 31. ...",40460,arrow media 40460


In [0]:
writer = pd.ExcelWriter('Sellout Matches Malaysia.xlsx')

matches_exact.to_excel(writer,sheet_name = 'Exact Matches',index = False)
semi_matches.to_excel(writer,sheet_name = '90% Matches',index = False)

writer.save()